#### 성능지표 - 분류
- 정확도 (Accuracy) : 전체 정답에서 맞춘 정답의 수
- 정밀도 (Precision) : 모델(예측) 기준으로 모델이 True라고 한것이 실제 True인 경우
- 재현율 (Recall) : 정답(실제) 기준으로 정답이 True라고 한것이 모델(예측) True인 경우
- F1 -Score : 정밀도와 재현율 2가지를 조합한 점수
- 오차행렬(Confusion Matrix) : 정답과 예측 값의 관계

[1] 모듈로딩

In [18]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score       #sklear에서 제공하는 모델 성능평가 관련 모듈

import pandas as pd
import numpy as np


[2] 데이터준비

In [19]:
digitBunch = load_digits(as_frame=True)

[3] 피쳐와 레이블 추출

In [20]:
# 0 ~ 9까지 이미지 데이터 값
imgDF=digitBunch['data']

In [21]:
# 0~ 9 라벨
labelSR = digitBunch['target']
labelSR.unique()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [22]:
labelSR2=(labelSR ==7).astype(int)
labelSR2.value_counts()

target
0    1618
1     179
Name: count, dtype: int64

In [23]:
### => 2진 분류를 위해서 7과 나머지 라벨 변경
mask=(labelSR==7)
labelSR[~mask] = 0
labelSR[mask] = 1

In [24]:
labelSR.value_counts()

target
0    1618
1     179
Name: count, dtype: int64

[4] 학습 준비

[4-1] 학습/테스트용 데이터셋 분리

In [26]:
X_train, X_test, y_train, y_test = train_test_split(imgDF,
                                                    labelSR,
                                                    stratify=labelSR,
                                                    random_state=1)

In [29]:
print('[Train]',y_train.value_counts() /y_train.shape[0])
print('[Test]', y_test.value_counts()/y_test.shape[0])

[Train] target
0    0.90052
1    0.09948
Name: count, dtype: float64
[Test] target
0    0.9
1    0.1
Name: count, dtype: float64


[4-2] 클래스 정의

In [30]:
# 사용자 정의 클래스 DummyClass : 아무것도 하지 않는 클래스, 단순 테스트용ㅇㅇ
from sklearn.base import BaseEstimator
class MyClassfier(BaseEstimator):
    def fit(self, X, y): pass

    def predict(self, X):
        return np.zeros( (len(X), 1), dtype=bool)
    

[5] 학습

In [31]:
# 인스턴스 생성
model = MyClassfier()

model.fit(X_train, y_train)

[6] 예측 및 성능 평가

In [34]:
pred = model.predict(X_train)
pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

[6-1] 정확도 => 불균형 데이터 경우 신뢰 불가!!

In [35]:
accuracy_score(y_train, pred)

0.9005196733481812

[6-2] 오차행렬

In [36]:
from sklearn.metrics import confusion_matrix, classification_report

In [38]:
# 정답과 예측값 전달
confusion_matrix(y_train, pred, labels=[0 ,1])

array([[1213,    0],
       [ 134,    0]], dtype=int64)

In [40]:
(tn, fp, fn, tp) = confusion_matrix(y_train, pred).reshape(-1)
print(tn, fp, fn, tp)

1213 0 134 0


- 정밀도 (Precision) : 모델(예측) 중심!!

In [41]:
from sklearn.metrics import precision_score

precision_score(y_train, pred, zero_division=0)

0.0

- 재현율(Recall) : 정답(실제) 중심!!

In [42]:
from sklearn.metrics import recall_score

recall_score(y_train,pred)

0.0

임의로 데이터를 만들어서 해봄

In [43]:
y_test = [0,0,0,1,1,  1,0,0,0,1,   1,1,1,0,0,    0,0,0,0,0]
y_pre =  [1,1,1,1,1,  1,0,1,0,1,   1,1,1,0,0,    0,0,0,0,0]

In [44]:
# 정답과 예측값 전달
(tn, fp, fn, tp) = confusion_matrix(y_test, y_pre).reshape(-1)
print(tn, fp, fn, tp)

9 4 0 7


In [45]:
confusion_matrix(y_test, y_pre)

array([[9, 4],
       [0, 7]], dtype=int64)

정밀도 : 모델이 True라고 분류한것 중에서 실제 True인것의 비율  
TP / (FP + TP)

In [52]:
# 정밀도

print(precision_score(y_test, y_pre, zero_division=0))
print(tp/(fp + tp))

0.6363636363636364
0.6363636363636364


재현율 : 실제 True인 것중에서 모델이 True라고 예측한것의 비율  
TP / (FN + TP)

In [54]:
# 재현률
print(recall_score(y_test,y_pre))
print(tp/(fn+tp))

1.0
1.0
